In [1]:
#general 
import pandas as pd
import numpy as np
import os
from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split

#warnings 
import warnings 
warnings.filterwarnings("ignore")

#for RNN
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model

import ast

Create Sentences for testing RNNS

In [2]:
word_df = pd.read_pickle("pos_tag_word_list")
sent_df = pd.read_csv("sample sent idea.csv")

Develop Benchmarks

In [3]:
def benchmark_string(sents):
    list_of_strings = []

    for word_list in sents:
        list_of_strings.append(' '.join(word_list))
    return list_of_strings

In [4]:
sent_df['list'] = sent_df['list'].apply(ast.literal_eval)
benchmark_sentences = sent_df['list'].tolist()
all_benchmarks = benchmark_string(benchmark_sentences)

In [5]:
loaded_model = tf.keras.models.load_model("C:/Users/DanielleDuncan/Desktop/THESIS/pickled_years/RNN_2011")

In [6]:
X_b = all_benchmarks
x_arrayb = np.asarray(X_b)
y_predb = loaded_model.predict(X_b)
Yb = pd.DataFrame(y_predb)
Yb.reset_index(inplace=True, drop=True)
Yb.set_axis(['0 Class Benchamark', '1 Class Benchmark'], axis='columns', inplace=True)
sent_df = pd.concat([sent_df, Yb], axis=1)

5/5 [==============================] - 3s 9ms/step


Test RNN model

#NOUNS

In [7]:
sent_df['replacement string'] = sent_df['replacement string'].apply(ast.literal_eval)


noun_list = word_df[(word_df.pos_tag_short == "N")]["feature"].values.tolist()
verb_list = word_df[(word_df.pos_tag_short == "V")]["feature"].values.tolist()
adverb_list = word_df[(word_df.pos_tag_short == "R")]["feature"].values.tolist()
adjective_list = word_df[(word_df.pos_tag_short == "J")]["feature"].values.tolist()

noun_sent = sent_df[(sent_df['POS tag'] == "Noun")]["replacement string"].values.tolist()
verb_sent = sent_df[(sent_df['POS tag'] == "Verb")]["replacement string"].values.tolist()
adjective_sent = sent_df[(sent_df['POS tag'] == "Adjective")]["replacement string"].values.tolist()
adverb_sent = sent_df[(sent_df['POS tag'] == "Adverb")]["replacement string"].values.tolist()

adverb_len = len(adverb_sent)
verb_len = len(verb_sent)
noun_len = len(noun_sent)
verb_len = len(verb_sent)

In [8]:
noun_len

74

In [16]:
def word_list(words, sents):
    new_token_sents = []
    for sent in sents:
        temp_sents = []
        for i, token in enumerate(sent):
            if token == 'UNK':
                for j, word in enumerate(words):
                    new_sent = list(sent)
                    new_sent[i] = word
                    temp_sents.append(' '.join(new_sent + [word]))
            else:
                temp_sents.append(' '.join(sent))
        new_token_sents.extend(temp_sents)
    return [(sent, sent.split()) for sent in new_token_sents]




adverb_sent = [['say', 'guy', 'hear', 'sing', 'think', 'moyles', 'year', 'UNK']]
adverb_list = ['friendly', 'just', 'sincere']

a = (word_list(adverb_list, adverb_sent))


In [17]:
a

[('say guy hear sing think moyles year UNK',
  ['say', 'guy', 'hear', 'sing', 'think', 'moyles', 'year', 'UNK']),
 ('say guy hear sing think moyles year UNK',
  ['say', 'guy', 'hear', 'sing', 'think', 'moyles', 'year', 'UNK']),
 ('say guy hear sing think moyles year UNK',
  ['say', 'guy', 'hear', 'sing', 'think', 'moyles', 'year', 'UNK']),
 ('say guy hear sing think moyles year UNK',
  ['say', 'guy', 'hear', 'sing', 'think', 'moyles', 'year', 'UNK']),
 ('say guy hear sing think moyles year UNK',
  ['say', 'guy', 'hear', 'sing', 'think', 'moyles', 'year', 'UNK']),
 ('say guy hear sing think moyles year UNK',
  ['say', 'guy', 'hear', 'sing', 'think', 'moyles', 'year', 'UNK']),
 ('say guy hear sing think moyles year UNK',
  ['say', 'guy', 'hear', 'sing', 'think', 'moyles', 'year', 'UNK']),
 ('say guy hear sing think moyles year friendly friendly',
  ['say',
   'guy',
   'hear',
   'sing',
   'think',
   'moyles',
   'year',
   'friendly',
   'friendly']),
 ('say guy hear sing think moyles

In [ ]:
def try_again(words, sents):
    RNN_strings=[]
    for sent in sents:
        for w in words:
            

In [9]:
def word_list(words, sents):
    new_token_sents = []
    for sent in sents:
        temp_sents = []
        for i, token in enumerate(sent):
            if token == 'UNK':
                for j, word in enumerate(words):
                    new_sent = list(sent)
                    new_sent[i] = word
                    temp_sents.append((' '.join(new_sent), sent))
            else:
                temp_sents.append((' '.join(sent), sent))
        new_token_sents.extend(temp_sents)
    return new_token_sents

sent_n = pd.DataFrame(word_list(noun_list, noun_sent))
sent_j = pd.DataFrame(word_list(adjective_list, adjective_sent))
sent_a = pd.DataFrame(word_list(adverb_list, adverb_sent))
sent_v = pd.DataFrame(word_list(verb_list, verb_sent))

In [ ]:
word_list(adverb_list, adverb_sent)


In [ ]:
adverb_list

In [10]:
sent_n

,0,1
0,sitting home watch UNK probably say,"[sitting, home, watch, UNK, probably, say]"
1,sitting home watch UNK probably say,"[sitting, home, watch, UNK, probably, say]"
2,sitting home watch UNK probably say,"[sitting, home, watch, UNK, probably, say]"
3,sitting home watch accompanies probably say,"[sitting, home, watch, UNK, probably, say]"
4,sitting home watch achieves probably say,"[sitting, home, watch, UNK, probably, say]"
...,...,...
47284,terrify UNK play well together also score twic...,"[terrify, UNK, play, well, together, also, sco..."
47285,terrify UNK play well together also score twic...,"[terrify, UNK, play, well, together, also, sco..."
47286,terrify UNK play well together also score twic...,"[terrify, UNK, play, well, together, also, sco..."
47287,terrify UNK play well together also score twic...,"[terrify, UNK, play, well, together, also, sco..."


In [ ]:
x_array_n = np.asarray(sent_n[0])
x_array_j = np.asarray(sent_j[0])
x_array_v = np.asarray(sent_v[0])
x_array_a = np.asarray(sent_a[0])


y_pred_n = loaded_model.predict(x_array_n)
y_pred_j = loaded_model.predict(x_array_j)
y_pred_v = loaded_model.predict(x_array_v)
y_pred_a = loaded_model.predict(x_array_a)

In [ ]:
#You need to change the num_repititons to the number of sentences in the function

def labels(word_list, word_len):
    num_repetitions = word_len
    repeated_words = []
    for word in word_list:
        for _ in range(num_repetitions):
            repeated_words.append(word)
    return repeated_words

noun_labels = pd.DataFrame(labels(noun_list, noun_len))
adjective_labels = pd.DataFrame(labels(adjective_list, noun_len))
verb_labels = pd.DataFrame(labels(verb_list, noun_len))
adverb_labels = pd.DataFrame(labels(adverb_list, noun_len))



In [ ]:
Y_n = pd.DataFrame(y_pred_n)
Y_j = pd.DataFrame(y_pred_j)
Y_v = pd.DataFrame(y_pred_v)
Y_a = pd.DataFrame(y_pred_a)

result_n = pd.concat([noun_labels, Y_n,sent_n], axis=1)
result_j = pd.concat([adjective_labels, Y_j,sent_j], axis=1)
result_v = pd.concat([verb_labels, Y_v,sent_v], axis=1)
result_a = pd.concat([adverb_labels, Y_a, sent_a], axis=1)

result_n.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_j.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_v.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_a.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)

In [ ]:
result = pd.concat([result_a, result_v, result_j, result_n])

In [ ]:
result_with_source = pd.merge(result, word_df, on='feature', how='inner')

In [ ]:
df = pd.merge(result_with_source, sent_df, on='replacement string', how='inner')

In [ ]:
df['Class 0 Differential'] = df['0 Class'] - df['0 Class Benchamark']
df['Class 1 Differential'] = df['1 Class'] - df['1 Class Benchmark']


In [ ]:
df

In [ ]:
df.groupby(by=['source']).mean()